## hide

In [ ]:
from geopy.distance import distance, geodesic
from geopy.point import Point
import math

def estimate_real_coords(drone_lat, drone_lon, altitude, heading, gimbal_pitch, x_pixel, y_pixel, image_width, image_height, hfov, vfov):
    # Calculate delta angles
    delta_azimuth = ((x_pixel - image_width / 2) / (image_width / 2)) * (hfov / 2)
    delta_elevation = ((image_height / 2 - y_pixel) / (image_height / 2)) * (vfov / 2)

    total_pitch = gimbal_pitch + delta_elevation
    if total_pitch >= 0 or total_pitch <= -120:
        raise ValueError(f"Invalid pitch: {total_pitch}°")

    # Normalize heading to 0-360 and compute total azimuth
    normalized_heading = heading % 360  # Converts -180-180 to 0-360
    total_azimuth = (normalized_heading + delta_azimuth) % 360

    # Calculate distance to target
    total_pitch_rad = math.radians(total_pitch)
    line_of_sight = altitude / math.sin(-total_pitch_rad)
    horizontal_distance = line_of_sight * math.cos(total_pitch_rad)

    # Compute new coordinates
    drone_point = Point(drone_lat, drone_lon)
    target_point = distance(meters=horizontal_distance).destination(drone_point, total_azimuth)
    return (target_point.latitude, target_point.longitude)

class TargetTracker:
    def __init__(self, distance_threshold=5.0):
        self.targets = []
        self.current_id = 0
        self.distance_threshold = distance_threshold  # meters

    def add_target(self, lat, lon):
        new_point = (lat, lon)
        min_dist = float('inf')
        min_id = None

        for target_id, t_lat, t_lon in self.targets:
            dist = geodesic((t_lat, t_lon), new_point).meters
            if dist < min_dist:
                min_dist = dist
                min_id = target_id

        if min_dist <= self.distance_threshold:
            return min_id
        else:
            self.targets.append((self.current_id, lat, lon))
            self.current_id += 1
            return self.current_id - 1

# Example Usage
if __name__ == "__main__":
    ## 131 34.03911397127907, -118.53399104254375
    drone_lat, drone_lon = 34.03930163888889, -118.53452375
    altitude = 80  # meters
    heading = 135  # degrees
    gimbal_pitch = -32  # degrees
    x_pixel, y_pixel = 732, 2819  # Image size 1600x1200
    hfov, vfov = 69.6, 55.2  # degrees

    # Estimate target coordinates
    target_lat, target_lon = estimate_real_coords(
        drone_lat, drone_lon, altitude, heading, gimbal_pitch,
        x_pixel, y_pixel, 4000, 3000, hfov, vfov
    )

    # Track targets
    tracker = TargetTracker(distance_threshold=5.0)
    target_id = tracker.add_target(target_lat, target_lon)
    print(f"Target Coordinates: {target_lat}, {target_lon} | ID: {target_id}")

Target Coordinates: 34.03911397127907, -118.53399104254375 | ID: 0


##test

In [ ]:

def main():
    tracker = TargetTracker(distance_threshold=5.0)
    all_images = [
        # ---------------- POS-1 ----------------
        {
            "pos_name": "Pos-1", "img_name": "0891",
            "drone_lat": 39.73789672222222, "drone_lon": -105.0775895555555,
            "altitude": 80, "heading": -179.9, "gimbal_pitch": -89.9,
            "x_pixel": 2282, "y_pixel": 2123,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-1", "img_name": "0896",
            "drone_lat": 39.7378975, "drone_lon": -105.0775902777778,
            "altitude": 80, "heading": -89.9, "gimbal_pitch": -59.9,
            "x_pixel": 2664, "y_pixel": 2764,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-1", "img_name": "0899",
            "drone_lat": 39.73789705555556, "drone_lon": -105.0775910555556,
            "altitude": 80, "heading": -44.9, "gimbal_pitch": -59.9,
            "x_pixel": 2242, "y_pixel": 2420,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-1", "img_name": "0903",
            "drone_lat": 39.73789808333333, "drone_lon": -105.0775898333333,
            "altitude": 80, "heading": 45.0, "gimbal_pitch": -59.9,
            "x_pixel": 1324, "y_pixel": 2921,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },

        # ---------------- POS-2 ----------------
        {
            "pos_name": "Pos-2", "img_name": "0836",
            "drone_lat": 39.73706794444445, "drone_lon": -105.0800276388889,
            "altitude": 80, "heading": 90.2, "gimbal_pitch": -59.9,
            "x_pixel": 2517, "y_pixel": 1752,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-2", "img_name": "0839",
            "drone_lat": 39.73706761111112, "drone_lon": -105.0800261111111,
            "altitude": 80, "heading": 134.9, "gimbal_pitch": -60.0,
            "x_pixel": 1528, "y_pixel": 1764,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-2", "img_name": "0847",
            "drone_lat": 39.73601733333334, "drone_lon": -105.0793547222222,
            "altitude": 80, "heading": -44.9, "gimbal_pitch": -32.0,
            "x_pixel": 3420, "y_pixel": 2020,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-2", "img_name": "0850",
            "drone_lat": 39.73601772222222, "drone_lon": -105.0793538333333,
            "altitude": 80, "heading": 0.0, "gimbal_pitch": -32.0,
            "x_pixel": 1553, "y_pixel": 1803,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },

        # ---------------- POS-3 ----------------
        {
            "pos_name": "Pos-3", "img_name": "0021",
            "drone_lat": 34.04031486111111, "drone_lon": -118.5352650833333,
            "altitude": 80, "heading": -44.8, "gimbal_pitch": -32.0,
            "x_pixel": 3451, "y_pixel": 2394,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-3", "img_name": "0022",
            "drone_lat": 34.04031472222222, "drone_lon": -118.5352650277778,
            "altitude": 80, "heading": -44.8, "gimbal_pitch": -59.9,
            "x_pixel": 3485, "y_pixel": 861,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-3", "img_name": "0023",
            "drone_lat": 34.04031505555555, "drone_lon": -118.5352648888889,
            "altitude": 80, "heading": -0.1, "gimbal_pitch": -59.9,
            "x_pixel": 1588, "y_pixel": 505,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-3", "img_name": "0053",
            "drone_lat": 34.04137161111111, "drone_lon": -118.5345688333333,
            "altitude": 80, "heading": -89.7, "gimbal_pitch": -59.9,
            "x_pixel": 1031, "y_pixel": 788,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },

        # ---------------- POS-4 ----------------
        {
            "pos_name": "Pos-4", "img_name": "0824",
            "drone_lat": 39.73706677777778, "drone_lon": -105.0800284444444,
            "altitude": 80, "heading": 179.7, "gimbal_pitch": -59.9,
            "x_pixel": 3858, "y_pixel": 1334,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-4", "img_name": "0826",
            "drone_lat": 39.73706641666666, "drone_lon": -105.0800280555556,
            "altitude": 80, "heading": -134.9, "gimbal_pitch": -32.0,
            "x_pixel": 2141, "y_pixel": 2241,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-4", "img_name": "0828",
            "drone_lat": 39.73706722222222, "drone_lon": -105.08002825,
            "altitude": 80, "heading": -89.8, "gimbal_pitch": -59.9,
            "x_pixel": 424, "y_pixel": 1136,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-4", "img_name": "0829",
            "drone_lat": 39.73706769444445, "drone_lon": -105.0800291111111,
            "altitude": 80, "heading": -89.8, "gimbal_pitch": -32.0,
            "x_pixel": 362, "y_pixel": 2682,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },

        # ---------------- POS-5 ----------------
        {
            "pos_name": "Pos-5", "img_name": "0635",
            "drone_lat": 28.07650508333333, "drone_lon": -82.46653719444444,
            "altitude": 80, "heading": 0.0, "gimbal_pitch": -60.0,
            "x_pixel": 3983, "y_pixel": 2136,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-5", "img_name": "0637",
            "drone_lat": 28.0765055, "drone_lon": -82.46653669444444,
            "altitude": 80, "heading": 45.3, "gimbal_pitch": -32.0,
            "x_pixel": 2697, "y_pixel": 2763,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-5", "img_name": "0638",
            "drone_lat": 28.07650552777778, "drone_lon": -82.46653669444444,
            "altitude": 80, "heading": 45.1, "gimbal_pitch": -60.0,
            "x_pixel": 2696, "y_pixel": 1210,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-5", "img_name": "0639",
            "drone_lat": 28.07650558333333, "drone_lon": -82.46653633333334,
            "altitude": 80, "heading": 90.0, "gimbal_pitch": -59.9,
            "x_pixel": 1275, "y_pixel": 1207,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },

        # ---------------- POS-6 ----------------
        {
            "pos_name": "Pos-6", "img_name": "0643",
            "drone_lat": 28.07621194444445, "drone_lon": -82.46539161111112,
            "altitude": 80, "heading": -179.9, "gimbal_pitch": -90.0,
            "x_pixel": 1720, "y_pixel": 51,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-6", "img_name": "0644",
            "drone_lat": 28.07621183333333, "drone_lon": -82.46539141666668,
            "altitude": 80, "heading": -179.9, "gimbal_pitch": -60.0,
            "x_pixel": 1774, "y_pixel": 1658,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-6", "img_name": "0647",
            "drone_lat": 28.07621186111111, "drone_lon": -82.4653915,
            "altitude": 80, "heading": -134.9, "gimbal_pitch": -59.9,
            "x_pixel": 816, "y_pixel": 2183,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
        {
            "pos_name": "Pos-6", "img_name": "0656",
            "drone_lat": 28.07621322222222, "drone_lon": -82.46539088888889,
            "altitude": 80, "heading": 90.0, "gimbal_pitch": -59.9,
            "x_pixel": 3545, "y_pixel": 2768,
            "image_width": 4000, "image_height": 3000,
            "hfov": 69.73, "vfov": 55.17
        },
    ]

    results = []

    for img_data in all_images:
        # Call the geometry function to compute lat/lon
        t_lat, t_lon = estimate_real_coords(
            drone_lat=img_data["drone_lat"],
            drone_lon=img_data["drone_lon"],
            altitude=img_data["altitude"],
            heading=img_data["heading"],
            gimbal_pitch=img_data["gimbal_pitch"],
            x_pixel=img_data["x_pixel"],
            y_pixel=img_data["y_pixel"],
            image_width=img_data["image_width"],
            image_height=img_data["image_height"],
            hfov=img_data["hfov"],
            vfov=img_data["vfov"]
        )

        # Add to tracker
        tid = tracker.add_target(t_lat, t_lon)

        # Store result
        results.append({
            "pos_name": img_data["pos_name"],
            "img_name": img_data["img_name"],
            "computed_lat": t_lat,
            "computed_lon": t_lon,
            "target_id": tid
        })

    # Print final
    print("=============== Final Results ===============")
    for r in results:
        print(
            f"Pos: {r['pos_name']} | Image: {r['img_name']} | "
            f"ID: {r['target_id']} | Computed: {r['computed_lat']:.12f}, {r['computed_lon']:.12f}"
        )

if __name__ == "__main__":
    main()


=============== Final Results ===============
Pos: Pos-1 | Image: 0891 | ID: 0 | Computed: 39.738040888995, -105.077573165938
Pos: Pos-1 | Image: 0896 | ID: 1 | Computed: 39.737915028621, -105.077700152475
Pos: Pos-1 | Image: 0899 | ID: 2 | Computed: 39.738025025752, -105.077733534311
Pos: Pos-1 | Image: 0903 | ID: 3 | Computed: 39.737939895245, -105.077554372462
Pos: Pos-2 | Image: 0836 | ID: 4 | Computed: 39.737013006595, -105.079588912503
Pos: Pos-2 | Image: 0839 | ID: 5 | Computed: 39.736865620236, -105.079674750836
Pos: Pos-2 | Image: 0847 | ID: 6 | Computed: 39.736789532172, -105.079682520240
Pos: Pos-2 | Image: 0850 | ID: 7 | Computed: 39.736949506623, -105.079476261782
Pos: Pos-3 | Image: 0021 | ID: 8 | Computed: 34.040924663186, -118.535496122299
Pos: Pos-3 | Image: 0022 | ID: 8 | Computed: 34.040925852424, -118.535516553803
Pos: Pos-3 | Image: 0023 | ID: 9 | Computed: 34.041120777821, -118.535388569951
Pos: Pos-3 | Image: 0053 | ID: 10 | Computed: 34.041178252568, -118.535348

In [ ]:
# from target_estimation import estimate_real_coords, TargetTracker
import math

if __name__ == "__main__":
    # Initialize target tracker
    tracker = TargetTracker()

    # # Pos-2 0836 39.73701311214281, -105.07958894136262
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73706794444445, -105.0800276388889
    # altitude = 80  # meters
    # heading = 90.2  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 2517, 1752  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # Pos-2 0839 39.736865571820985, -105.07967486297535
    # 39.73686562023648, -105.07967475083579
    # Initialize drone parameters
    # drone_lat, drone_lon = 39.73706761111112, -105.0800261111111
    # altitude = 80  # meters
    # heading = 134.9  # degrees
    # gimbal_pitch = -60  # degrees
    # x_pixel, y_pixel = 1528, 1764  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-2 0847 39.73678022937951, -105.0797172265706
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73601733333334, -105.0793547222222
    # altitude = 80  # meters
    # heading = -44.9  # degrees
    # gimbal_pitch = -32  # degrees
    # x_pixel, y_pixel = 3420, 2020  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-2 0850 39.736945640212674, -105.07951830715356
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73601772222222, -105.0793538333333
    # altitude = 80  # meters
    # heading = 0  # degrees
    # gimbal_pitch = -32  # degrees
    # x_pixel, y_pixel = 1553,  1803  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-1 0891 39.73804088899502, -105.07757316593765
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73789672222222, -105.0775895555555
    # altitude = 80  # meters
    # heading = -179.9  # degrees
    # gimbal_pitch = -89.9  # degrees
    # x_pixel, y_pixel = 2282, 2123  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-1 0896 39.73791502862122, -105.07770015247475
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.7378975, -105.0775902777778
    # altitude = 80  # meters
    # heading = -89.9  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 2664, 2764  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-1 0899 39.73802502575232, -105.07773353431065
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73789705555556, -105.0775910555556
    # altitude = 80  # meters
    # heading = -44.9  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 2242, 2420  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # Pos-1 0903 39.73793989524488, -105.07755437246217
    # Initialize drone parameters
    drone_lat, drone_lon = 39.73789808333333, -105.0775898333333
    altitude = 80  # meters
    heading = 45  # degrees
    gimbal_pitch = -59.9  # degrees
    x_pixel, y_pixel = 1324, 2921  # Image size 1600x1200
    hfov, vfov = 69.73, 55.17  # degrees
    image_width, image_height = 4000, 3000

    # # Pos-3 0021 34.040917579314474, -118.53552154609152
    # # Initialize drone parameters
    # drone_lat, drone_lon = 34.04031486111111, -118.5352650833333
    # altitude = 80  # meters
    # heading = -44.8  # degrees
    # gimbal_pitch = -32  # degrees
    # x_pixel, y_pixel = 3451, 2394  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-3 0022 34.04092585242436, -118.53551655380284
    # # Initialize drone parameters
    # drone_lat, drone_lon = 34.04031472222222, -118.5352650277778
    # altitude = 80  # meters
    # heading = -44.8  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 3485, 861  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-3 0023 34.04112077782075, -118.53538856995128
    # # Initialize drone parameters
    # drone_lat, drone_lon = 34.04031505555555, -118.5352648888889
    # altitude = 80  # meters
    # heading = -0.1  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 1588, 505  # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-3 0053 34.04117825256812, -118.5353483534665
    # # Initialize drone parameters
    # drone_lat, drone_lon = 34.04137161111111, -118.5345688333333
    # altitude = 80  # meters
    # heading = -89.7  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 1031, 788 # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-4 0824 39.736668031894666, -105.0803522963074
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73706677777778, -105.0800284444444
    # altitude = 80  # meters
    # heading = 179.7  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 3858, 1334 # Image size 1600x1200
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-4 0826 39.73659069761638, -105.08070185374521
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73706641666666, -105.0800280555556
    # altitude = 80  # meters
    # heading = -134.9  # degrees
    # gimbal_pitch = -32  # degrees
    # x_pixel, y_pixel = 2141, 2241
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-4 0828 39.73682027213527, -105.08064866652333
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73706722222222, -105.08002825
    # altitude = 80  # meters
    # heading = -89.8  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 424, 1136
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-4 0829 39.73681666426292, -105.0806315971058
    # # Initialize drone parameters
    # drone_lat, drone_lon = 39.73706769444445, -105.0800291111111
    # altitude = 80  # meters
    # heading = -89.8  # degrees
    # gimbal_pitch = -32  # degrees
    # x_pixel, y_pixel = 362, 2682
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-5 0635 28.076701719594567, -82.46638443277486
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07650508333333, -82.46653719444444
    # altitude = 80  # meters
    # heading = 0  # degrees
    # gimbal_pitch = -60  # degrees
    # x_pixel, y_pixel = 3983, 2136
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-5 0637 28.076775174319057, -82.46606034295554
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.0765055, -82.46653669444444
    # altitude = 80  # meters
    # heading = 45.3  # degrees
    # gimbal_pitch = -32  # degrees
    # x_pixel, y_pixel = 2697, 2763
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-5 0638 28.076782498558124, -82.46605152411313
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07650552777778, -82.46653669444444
    # altitude = 80  # meters
    # heading = 45.1  # degrees
    # gimbal_pitch = -60  # degrees
    # x_pixel, y_pixel = 2696, 1210
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-5 0639 28.07661819721678, -82.46597009645227
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07650558333333, -82.46653633333334
    # altitude = 80  # meters
    # heading = 90  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 1275, 1207
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-6 0643 28.07585096812442, -82.4653575709036
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07621194444445, -82.46539161111112
    # altitude = 80  # meters
    # heading = -179.9  # degrees
    # gimbal_pitch = -90  # degrees
    # x_pixel, y_pixel = 1720, 51
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-6 0644 28.075843343492018, -82.46536353269025
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07621183333333, -82.46539141666668
    # altitude = 80  # meters
    # heading = -179.9  # degrees
    # gimbal_pitch = -60  # degrees
    # x_pixel, y_pixel = 1774, 1658
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-6 0647 28.07600417894965, -82.46549801158157
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07621186111111, -82.46539150000001
    # altitude = 80  # meters
    # heading = -134.9  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 816,2183
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # # Pos-6 0656 28.076174339401913, -82.46530460214758
    # # Initialize drone parameters
    # drone_lat, drone_lon = 28.07621322222222, -82.46539088888889
    # altitude = 80  # meters
    # heading = 90  # degrees
    # gimbal_pitch = -59.9  # degrees
    # x_pixel, y_pixel = 3545, 2768
    # hfov, vfov = 69.73, 55.17  # degrees
    # image_width, image_height = 4000, 3000

    # Estimate target coordinates
    target_lat, target_lon = estimate_real_coords(
        drone_lat, drone_lon, altitude, heading, gimbal_pitch, x_pixel, y_pixel,
        image_width, image_height, hfov, vfov
    )

    # Add target to tracker
    target_id = tracker.add_target(target_lat, target_lon)
    print(f"Target ID: {target_id}, Latitude and Longitude: {target_lat}, {target_lon}")

Target ID: 0, Latitude and Longitude: 39.73794082522682, -105.07755628413793


In [ ]:
!pip install folium

##week22 david

## week23 data


In [ ]:
import folium

coordsDavid = [
    # # --- position1 (David) ---

    (39.73804089, -105.0775732),  # img_1
    (39.73791503, -105.0777002),  # img_2
    (39.73802503, -105.0777335),  # img_3
    (39.7379399,  -105.0775544),  # img_4
    (39.738028, -105.077672),     # target

    # --- position2 (David) ---

    (39.73701311, -105.0795889),  # img_1
    (39.73686557, -105.0796749),  # img_2
    (39.73678023, -105.0797172),  # img_3
    (39.73694564, -105.0795183),  # img_4
    (39.736929,   -105.079601),   # target

    # --- position3 (David) ---

    (34.04091758,  -118.5355215), # img_1
    (34.04092585,  -118.5355166), # img_2
    (34.04112078,  -118.5353886), # img_3
    (34.04117825,  -118.5353484), # img_4
    (34.0411,      -118.535355),  # target

    # --- position4 (David) ---

    (39.73666803, -105.0803523),  # img_1
    (39.7365907,  -105.0807019),  # img_2
    (39.73682027, -105.0806487),  # img_3
    (39.73681666, -105.0806316),  # img_4
    (39.736659,   -105.080599),   # target

    # --- position5 (David) ---

    (28.07670172, -82.46638443),  # img_1
    (28.07677517, -82.46606034),  # img_2
    (28.0767825,  -82.46605152),  # img_3
    (28.0766182,  -82.4659701),   # img_4
    (28.076706,   -82.46598),     # target

    # --- position6 (David) ---

    (28.07585097, -82.46535757),  # img_1
    (28.07584334, -82.46536353),  # img_2
    (28.07600418, -82.46549801),  # img_3
    (28.07617434, -82.4653046),   # img_4
    (28.075868,   -82.465298),    # target

]

coordsTracy = [
    # --- Position 1 (4 images) ---
    (39.738040888995, -105.077573165938),  # Pos-1, Image: 0891
    (39.737917243148, -105.077699520497),  # Pos-1, Image: 0896
    (39.738028786723, -105.077727662722),  # Pos-1, Image: 0899
    (39.737940366676, -105.077555323044),  # Pos-1, Image: 0903

    # --- Position 2 (4 images) ---
    (39.737018926425, -105.079587737421),  # Pos-2, Image: 0836
    (39.736870385280, -105.079670238295),  # Pos-2, Image: 0839
    (39.736789532172, -105.079682520240),  # Pos-2, Image: 0847
    (39.736949506623, -105.079476261782),  # Pos-2, Image: 0850

    # --- Position 3 (4 images) ---
    (34.040924663186, -118.535496122299),  # Pos-3, Image: 0021
    (34.040932787744, -118.535490780361),  # Pos-3, Image: 0022
    (34.041123880308, -118.535354716480),  # Pos-3, Image: 0023
    (34.041195305722, -118.535354166567),  # Pos-3, Image: 0053

    # --- Position 4 (4 images) ---
    (39.736666944732, -105.080350039690),  # Pos-4, Image: 0824
    (39.736604478051, -105.080717752414),  # Pos-4, Image: 0826
    (39.736832895337, -105.080656826974),  # Pos-4, Image: 0828
    (39.736828926488, -105.080639902035),  # Pos-4, Image: 0829

    # # --- Position 5 (4 images) ---
    # (28.076696871212, -82.466376788619),   # Pos-5, Image: 0635
    # (28.076782506880, -82.466065721858),   # Pos-5, Image: 0637
    # (28.076789966518, -82.466057047930),   # Pos-5, Image: 0638
    # (28.076626945070, -82.465972398564),   # Pos-5, Image: 0639

        # --- Position 5 (4 images) ---
    (28.076704054304, -82.466388325238),   # Pos-5, Image: 0635
    (28.07678250688, -82.466065721858),   # Pos-5, Image: 0637
    (28.076789966518, -82.46605704793),   # Pos-5, Image: 0638
    (28.07662694507, -82.465972398564),   # Pos-5, Image: 0639


    # --- Position 6 (4 images) ---
    (28.075850968124, -82.465357570904),   # Pos-6, Image: 0643
    (28.075843454912, -82.465361720196),   # Pos-6, Image: 0644
    (28.076006723070, -82.465504104477),   # Pos-6, Image: 0647
    (28.076175680994, -82.465303850198)    # Pos-6, Image: 0656
]

# Function to extract a position slice (each position has 5 coordinates)
def get_position_slice(coords, pos_name,items_per_position):
    """
    Given the coords list and pos_name (e.g., "Pos-1"), return the slice of 5 coordinates.
    """
    # Assume pos_name format "Pos-X" where X is 1-indexed
    pos_number = int(pos_name.split("-")[1])
    start = (pos_number - 1) * items_per_position
    end = start + items_per_position
    return coords[start:end]

## Compare

In [ ]:
from folium.plugins import MarkerCluster

desired_position = "Pos-5"
david_slice = get_position_slice(coordsDavid, desired_position, items_per_position=5)
tracy_slice = get_position_slice(coordsTracy, desired_position, items_per_position=4)

image_names = ["image1", "image2", "image3", "image4","target"]
center_lat, center_lon = david_slice[0]
m = folium.Map(location=[center_lat, center_lon], zoom_start=20)
# marker_cluster = MarkerCluster().add_to(m)
offsets = [0, 0.00001, -0.00001, 0.00002, -0.00002]
for i, (lat, lon) in enumerate(david_slice):
    color = "red" if i == 4 else "blue"
    folium.Marker(
        location=[lat, lon],
        popup=f"David {image_names[i]}",
        icon=folium.Icon(color=color)
    ).add_to(m)

for i, (lat, lon) in enumerate(tracy_slice):
    folium.Marker(
        location=[lat + offsets[i], lon + offsets[i]],
        popup=f"Tracy {image_names[i]}",
        icon=folium.Icon(color="green")
    ).add_to(m)

file_name = f"map_{desired_position}.html"
m.save(file_name)
m